# Miniproject 2

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
from typing import (Callable, Iterable, List, Sequence, Optional, Dict)

from abc import abstractmethod, ABCMeta
import sys
import math
import random
import dataclasses
from collections import namedtuple

import numpy as np

import scipy.stats
import scipy.special

# Basic data structures.

# x, y coordinate and the rotation of the robot.
Pose = namedtuple("Pose", ["x", "y", "theta"])
# unique id (str) and x, y coordinate of the landmark.
Landmark = namedtuple("Landmark", ["id", "x", "y"])
# r --- distance to the landmark; b --- angle to the landmark, computed by arctan2.
Measurement = namedtuple("Measurement", ["landmark_id", "r", "b"])

Command = namedtuple("Command", ["delta_p", "delta_theta"])


def grid_of_landmarks(
        x_range: Iterable[float] = range(-4, 20, 4),
        y_range: Iterable[float] = range(-4, 20, 4),
) -> Sequence[Landmark]:
    """Constructs a grid of landmarks from a catesion product of x and y coordinate ranges."""
    return tuple(
        Landmark(str(i), *loc)
        for i, loc in enumerate(itertools.product(x_range, y_range)))


class Inference(metaclass=ABCMeta):
    """Interface for an inference algorithm (localization or SLAM variants)."""

    @abstractmethod
    def estimated_pose(self) -> Pose:
        """Returns the current estimation of the robot's Pose."""
        ...

    @abstractmethod
    def estimated_landmarks(self) -> Sequence[Landmark]:
        """Returns the current estimation of the landmarks."""
        ...

    @abstractmethod
    def init(self, init_state: Pose) -> None:
        """Initialize the inference with an initial pose."""
        ...

    @abstractmethod
    def update(self, measurements: Sequence[Measurement]) -> None:
        """An update step of the inference, called after every simulation step."""
        ...

    @abstractmethod
    def plot_state(self, ax) -> None:
        """Helper to visualize the internal state of the inference algorithm."""
        ...


def normalize_angles(angles: np.array) -> np.ndarray:
    """Given an array of angles in radians, element-wise normalize the angles to the range [-pi, pi].

    Args:
        angles: array of any shape.

    Returns:
        normalized angles, array of same shape as the input.
    """
    return np.arctan2(np.sin(angles), np.cos(angles))


@dataclasses.dataclass(frozen=True)
class SimulationResult:
    """A helper class to hold the results of running a simulation."""
    sim: 'Simulator'  # Simulator that generated this result
    infer: Optional[Inference]  # Inference procedure that generated this result
    true_poses: Sequence[Pose]
    estimated_poses: Optional[Sequence[Pose]] = None
    # History snapshots for the Inference, for animation purpose
    snapshots: Optional[Sequence[Inference]] = None


@dataclasses.dataclass(frozen=False)
class Simulator:
    """A simulator for a point robot in R^2.

    The simulator simulates the motion and sensor noises of the robot by 
    sampling from 2D Gaussians.
    The simulation has a list of landmarks, each with a unique identifier.
    The sensor of the robot can sense, for each landmark in the robot's sensing
    range, a noise-corrupted distance and bearing to that landmarks (together
    with the identifier that identifies the landmark).
    """

    # Initial simulator state
    init_state: Pose = Pose(0, 0, 0)

    # 2x2 covariance matrix of the motion noise.
    motion_noise_covariance: np.ndarray = np.diag([1e-3, np.deg2rad(3)**2])

    # 2x2 covariance matrix of the sensor noise.
    sensor_noise_covariance: np.ndarray = np.diag([1, np.deg2rad(5)**2])

    # A set of landmarks
    landmarks: Sequence[Landmark] = grid_of_landmarks()

    # Robot can only sense landmarks within this range
    max_sensing_range: float = np.inf

    # Current state of the robot
    state: Pose = dataclasses.field(init=False, default=None)

    # Random seed control
    rng: np.random.Generator = dataclasses.field(init=False, default=None)
    # Using the same seed will induce the same trajectory
    seed: dataclasses.InitVar[int] = 0

    def __post_init__(self, seed: int):
        assert (len({l.id for l in self.landmarks
                    }) == len(self.landmarks)), "Landmark must have unique IDs"
        self.rng = np.random.default_rng(seed)

    def init(self) -> None:
        self.state = self.init_state

    def simulate_motion(self, command: Command) -> None:
        """Simulate robot motion. This function updates the value of
        self.state.

        Args:
          command: a Command tuple containing fields delta_p(float), the 
          distance of the movement, and delta_theta(float), the rotation of 
          the movement.
        """
        noise = self.rng.multivariate_normal(np.zeros(2),
                                             self.motion_noise_covariance)
        x = (self.state.x + math.cos(self.state.theta) *
             (command.delta_p + noise[0]))
        y = (self.state.y + math.sin(self.state.theta) *
             (command.delta_p + noise[0]))
        theta = self.state.theta + command.delta_theta + noise[1]
        self.state = Pose(x, y, theta)

    def simulate_sensing(self) -> Sequence[Measurement]:
        """Simulate the robot sensing process. This function returns a list of
        measurements, in the same order as the self.landmarks list.
        Specifically, it measures the distance and the angle w.r.t. each
        landmark, and add a Gaussian noise on the measurement.

        Returns:
          measurements: a list of measurements.
        """
        measurements = []
        # Simulate that the measurements' order is random
        for landmark in sorted(self.landmarks, key=lambda _: random.random()):
            r = math.hypot(landmark.x - self.state.x, landmark.y - self.state.y)

            if r >= self.max_sensing_range:
                # Can't sense this landmark, skip
                continue

            b = math.atan2(landmark.y - self.state.y,
                           landmark.x - self.state.x) - self.state.theta
            # Normalize angle to -pi to pi
            b = math.atan2(math.sin(b), math.cos(b))

            noise = self.rng.multivariate_normal(np.zeros(2),
                                                 self.sensor_noise_covariance)

            measurements.append(
                Measurement(landmark.id, r + noise[0], b + noise[1]))

        return measurements

    def run(self,
            commands: Iterable[Command],
            infer: Optional[Inference] = None,
            snapshot_every_n: Optional[int] = 0) -> SimulationResult:
        """Run the simulation with a sequence of commands. Optionally, execute
        an inference algorithm along way with the simulation.

        Args:
            commands: an interable of commands.
            infer: the inference algorithm to run the simulation with.
            snapshot_every_n: snapshot the inference algorithm every n steps.
                if <= 0, don't take any snapshot. 
        Returns:
            SimulationResult
        """
        true_poses, est_poses, snapshots = [], None, None

        self.init()

        i = 0
        true_poses.append(self.state)
        if infer:
            est_poses, snapshots = [], []
            infer.init(self.init_state)
            est_poses.append(infer.estimated_pose())
            if snapshot_every_n > 0 and i % snapshot_every_n == 0:
                snapshots.append(copy.deepcopy(infer))

        for command in commands:
            i += 1
            # Simulate motion
            self.simulate_motion(command)
            true_poses.append(self.state)
            if infer:
                # Simulate measurement
                measurements = self.simulate_sensing()
                infer.update(command, measurements)
                est_poses.append(infer.estimated_pose())
                if snapshot_every_n > 0 and i % snapshot_every_n == 0:
                    snapshots.append(copy.deepcopy(infer))

        return SimulationResult(self, copy.deepcopy(infer), true_poses,
                                est_poses, snapshots)


def drive_in_a_square_commands() -> Iterable[Command]:
    """Commands to simulate the robot's movement in a square."""
    for _ in range(4):
        for i in range(100):
            yield Command(.1, 0)
        yield Command(0, np.pi / 2)


def patrol_commands() -> Iterable[Command]:
    """Commands that walks back and forth horizontally."""
    for i in range(10):
        yield Command(0.5, 0)
    yield Command(0, np.pi)





## Warmup 1


### Question
Let's make sure we know how to sample from a set 
of values based on their weights.

Please write a function that takes in a list of samples and associated weights,
and re-samples from the list of samples with replacement according the
likelihood of each sample given by the list of weights.

This should be an easy question --- you should feel free to use the sampling
helper functions in numpy. Specifically, take a look at the function 
`numpy.random.choice`.


For reference, our solution is **2** line(s) of code.

In [ ]:
def warmup_1(samples: np.ndarray, weights: np.ndarray,
             nr_samples: int) -> np.ndarray:
    """Draw N samples from the input list based on their weights.

    Args:
      samples: a numpy array of shape (n_samples, ), a list of samples.
      weights: a numpy float array of shape (n_samples, ), indicating 
        the weights for individual samples.
      nr_samples: an integer, indicating the number of samples to draw.

    Returns:
      resampled_samples: the return numpy array, containing nr_samples
        samples drawn from the input list, based on the weights with replacement.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(warmup_1(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=np.int64), np.array([0.03191091, 0.10365558, 0.07293406, 0.13085621, 0.12995933, 0.0454193 , 0.04606439, 0.1336077 , 0.15964489, 0.14594763], dtype=np.float64), 10), np.array([6, 8, 7, 6, 4, 7, 4, 9, 9, 4], dtype=np.int64), atol=1e-6)
print('Tests passed.')

## Warmup 2


### Question
We are going to be sampling from Gaussian
distributions in this miniproject. Let us check that we know how to sample from a
multi-variate Gaussian.

Please write a function that takes in a mean vector, a covariance matrix and a
integer number of samples and returns a list of the requested number of
samples drawn from a Gaussian with the given mean and covariance.

This should be an easy question --- you should feel free to use the sampling
helper functions in numpy. Specifically, take a look at the function numpy.random.multivariate_normal.


For reference, our solution is **2** line(s) of code.

In [ ]:
def warmup_2(mean: np.ndarray, covariance: np.ndarray,
             num_particles: int) -> np.ndarray:
    """Sample from a multi-variate Gaussian distribution.

    Args:
      mean: a numpy array of shape (N, ).
      covariance: a numpy array of shape (N, N).
      num_particles: an integer.

    Returns:
      sample: a numpy vector with length num_particles.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(warmup_2(np.array([0.58345008, 0.05778984, 0.13608912], dtype=np.float64), np.array([[1.61405906, 0.93399482, 0.79165733],  [0.93399482, 1.65599579, 1.43616931],  [0.79165733, 1.43616931, 1.25177385]], dtype=np.float64), 5), np.array([[-0.8347847 , -2.30256418, -1.83805061],  [-0.13115218, -3.33917428, -2.91354857],  [-0.47476854, -1.05885092, -0.83435982],  [ 0.29510331, -0.55660856, -0.28675913],  [-0.06919861, -0.94320128, -0.69794801]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

## Localization: Update by Motion Model


### Utilities

Particle Filter for Localization
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:

try:
    from functools import cached_property
except ImportError:
    # Import for Colab (Python==3.7)
    from backports.cached_property import cached_property


def multivariate_normal_logpdf(xs: np.ndarray, means: np.ndarray,
                               covs: np.ndarray) -> np.ndarray:
    """Batch version of `scipy.stats.multivariate_normal.logpdf`.

    Compute multivariate_normal.logpdf, with batch inputs and distributions.
    Modified from https://gregorygundersen.com/blog/2020/12/12/group-multivariate-normal-pdf/. 

    This function uses the following broadcasting logic:
    - First, `means` and `covs` are broadcasted together, which determines `dist_batch`, 
        the number of individual multivariate normal distributions. 
    - Then, we check that `xs` is broadcastable to `means`.

    Args:
        xs: array of shape (...input_batch,  ...dist_batch, dist_dim)
        means: array of shape (...dist_batch, dist_dim)
        covs: array of shape (...dist_batch, dist_dim, dist_dim)

    Returns:
        array of shape (...input_batch, ...dist_dim)
    """
    dist_dim = xs.shape[-1]
    dist_batch_shape = np.broadcast_shapes(means.shape[:-1], covs.shape[:-2])
    means = np.broadcast_to(means, dist_batch_shape + (dist_dim,))
    covs = np.broadcast_to(covs, dist_batch_shape + (dist_dim, dist_dim))
    xs = np.broadcast_to(xs, xs.shape[:-len(means.shape)] + means.shape)

    vals, vecs = np.linalg.eigh(covs)
    logdets = np.sum(np.log(vals), axis=-1)
    valsinvs = 1. / vals
    Us = vecs * np.sqrt(valsinvs)[..., np.newaxis, :]
    devs = xs - means
    devUs = np.einsum("...i,...ij->...j", devs, Us)
    mahas = np.sum(np.square(devUs), axis=-1)
    log2pi = np.log(2 * np.pi)
    out = -0.5 * (dist_dim * log2pi + mahas + logdets)
    return out


@dataclasses.dataclass(frozen=True)
class LocalizationParticles:
    """A batch of particles."""

    x: np.ndarray  # shape (N, )
    y: np.ndarray  # shape (N, )
    theta: np.ndarray  # shape (N, )

    def __post_init__(self):
        assert self.x.shape == self.y.shape == self.theta.shape
        assert len(self.x.shape) == 1
        # Always normalize the angles
        object.__setattr__(self, "theta", normalize_angles(self.theta))

    def __len__(self) -> int:
        """The number of particles."""
        return self.x.shape[0]


@dataclasses.dataclass
class Localization(Inference):
    """Localization Inference by particle filtering."""

    # A 2x2 array for the motion covariance.
    motion_noise_covariance: np.ndarray

    # A 2x2 numpy array for the sensor noise for the
    # measurements of range and bearing to the landmarks
    sensor_noise_covariance: np.ndarray

    # In localization, we assume that we have access to the set of landmarks.
    landmarks: List[Landmark] = dataclasses.field(default_factory=list)

    num_particles: int = 10

    particles: LocalizationParticles = None

    def __post_init__(self):
        if self.particles:
            assert len(self.particles) == self.num_particles

    @cached_property
    def landmarks_id_map(self) -> Dict[str, Landmark]:
        """Map from unique landmark id to the Landmark instance."""
        return {l.id: l for l in self.landmarks}

    def estimated_pose(self) -> Pose:
        return Pose(
            *map(np.mean,
                 [self.particles.x, self.particles.y, self.particles.theta]))

    def estimated_landmarks(self) -> Sequence[Landmark]:
        # we know the landmarks so no estimation required!
        return self.landmarks

    def init(self, init_state: Pose) -> None:
        self.particles = LocalizationParticles(
            np.full(self.num_particles, init_state.x, dtype=np.float64),
            np.full(self.num_particles, init_state.y, dtype=np.float64),
            np.full(self.num_particles, init_state.theta, dtype=np.float64),
        )

    def plot_state(self, ax: 'matplotlib.axes.Axes') -> None:
        ax.quiver(
            self.particles.x,
            self.particles.y,
            np.cos(self.particles.theta),
            np.sin(self.particles.theta),
            angles="xy",
            scale=100,
            color=(1.0, 0, 0, 0.4),
        )

    # We incrementally implement methods below using the `implementation_for` helper
    def motion_model(self, command: Command):
        raise NotImplementedError()

    def compute_weights(self,
                        measurements: Sequence[Measurement]) -> np.ndarray:
        raise NotImplementedError()

    def update(self, command: Command, measurements: Sequence[Measurement]):
        raise NotImplementedError()

### Question

Let us now start by building a motion model for a
simple ground robot that moves in straight lines, and performs point turns.

A common motion model for such a robot assumes that the robot has a pose given
by $[x_t, y_t, \theta_t]$ at time $t$. The robot takes a command $[\Delta p_t,
\Delta \theta_t]$ which modifies the robot's pose according to the following:

$$
\begin{bmatrix} x_{t+1} \\ y_{t+1} \\ \theta_{t+1} \end{bmatrix} =
\begin{bmatrix} x_t + \cos(\theta_t) (\Delta p_t + \omega_t) \\
y_t + \sin(\theta_t)(\Delta p_t + \omega_t) \\
\theta_t + \Delta \theta_t + \nu_t
\end{bmatrix}
$$

where $[\omega_t, \nu_t]$ is Gaussian error injected at each time
step, distributed as $\omega_t \sim \mathcal{N}(0, \sigma^2_p)$, and
$\nu_t \sim \mathcal{N}(0, \sigma^2_\theta)$. The units of $\Delta  p_t$ is in
metres and $\Delta \theta_t$ in radians.

Let us assume that the robot's pose at each time step is a random variable,
represented as a set of samples.

Please implement a function that takes in a set of sampled poses
drawn from prior pose random variable $\mathbf{X}_t$
and a control $[\Delta p_t, \Delta \theta_t]$ and returns a set
of samples drawn from the posterior $X_{t+1}$.

**Note**
In this project, we will use a helper decorator `implementation_for` to implement
class methods outside of the class. 
The decorator `@implementation_for(Localization)` simply inserts the declared
function as a method of the Localization class. 
Using this decorator, you copy only the implemented method into 
catsoop's solution box and avoid repeating other parts of the class definition.


**Hint**: Use `np.random.multivariate_normal` to generate the noise.


For reference, our solution is **10** line(s) of code.

In [ ]:
@implementation_for(Localization)
def motion_model(self: Localization, command: Command) -> None:
    """A motion model that simulates a one-step movement of the robot, and
    updates to the new samples.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
            movement, and delta_theta(float), the rotation of the movement.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def localization_motion_model_problem_test():
    # A simulator with no noise
    sim = Simulator(motion_noise_covariance=np.zeros((2, 2)),
                    sensor_noise_covariance=np.zeros((2, 2)))
    sim.init()
    # Particle filter with implemented motion_model
    num_particles = 10
    pf = Localization(
        motion_noise_covariance=[[.01, 0], [0, np.deg2rad(5)**2]],
        sensor_noise_covariance=sim.sensor_noise_covariance,
        landmarks=sim.landmarks,
        num_particles=num_particles,
    )
    pf.init(sim.state)
    command = Command(1, 0)
    assert pf.motion_model(command) is None, "update should be in-place"
    assert len(pf.particles) == num_particles
    sim.simulate_motion(command)
    pose = sim.state
    est_pose = pf.estimated_pose()
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=0.1
    ), "Estimated pose should be quite close to simulation without noise"

localization_motion_model_problem_test()

print('Tests passed.')

## Localization: Computing the Importance Weights


### Question

Let us now add in a measurement model. Let us assume that
there are four landmarks located at $(x, y)$ locations (-1, -1), (11, -1), (11, 11) and
(-1, 11). The robot is equipped with a sensor that can measure the range and
relative bearing $[r, b]$ from the robot's location to each of these landmarks. If the
robot's pose is $(x, y, \theta)$ and the landmark position is $(l_x, l_y)$,
then range is just the Euclidean distance $$r = \sqrt{(l_x - x)^2 + (l_y
-y)^2}$$, 
and the bearing is the angle to the landmark in the robot's body frame (hence
relative bearing), $$b = \texttt{atan2}(l_y - y, l_x - x) - \theta.$$  Each of
these measurements is corrupted with noise $[q_r, q_b]$ distributed according
to $q_r \sim \mathcal{N}(0, \sigma^2_r)$ and $q_b \sim \mathcal{N}(0,
\sigma^2_b)$.

Please take a look at our implementation in `simulator.simulate_sensing` and
understand how these values are computed.

Now the problem is that for a real robot, even if you know this is the model
that generates the measurements, you can't know the noise that is being
applied. (That would be omniscience!) What we can do is work out how likely
each measurement is.  More precisely, given a set of measurements, we can use
this model to compute the importance weights of a set of samples of the robot
pose. See [Lecture 9 Page 13](https://sicp-s4.mit.edu/_static/fall22/lectures/lec09/L09-Sampling.pdf#page=13)
for details.

Please implement a function that takes a set of samples and measurements and
returns the importance weights of the samples.


For reference, our solution is **28** line(s) of code.

In [ ]:
@implementation_for(Localization)
def compute_weights(self: Localization,
                    measurements: Sequence[Measurement]) -> np.ndarray:
    """Compute the importance weights of the samples, based on the new
    measurement.

    Args:
        measurements: a sequence of measurements made by the robot.

    Returns:
        weights: a numpy array of importance weights, normalized to 1.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def sampling_problem2_test(compute_weights, infer: Localization,
                           measurements: Sequence[Measurement],
                           result_weights: np.ndarray):
    weights = compute_weights(infer, measurements)
    assert np.allclose(weights, result_weights, atol=1e-4)

sampling_problem2_test(compute_weights, Localization(motion_noise_covariance=[[0.01, 0], [0, 0.007615435494667714]], sensor_noise_covariance=array([[1.        , 0.        ],
       [0.        , 0.00761544]]), landmarks=(Landmark(id="0", x=-4, y=-4), Landmark(id="1", x=-4, y=0), Landmark(id="2", x=-4, y=4), Landmark(id="3", x=-4, y=8), Landmark(id="4", x=-4, y=12), Landmark(id="5", x=-4, y=16), Landmark(id="6", x=0, y=-4), Landmark(id="7", x=0, y=0), Landmark(id="8", x=0, y=4), Landmark(id="9", x=0, y=8), Landmark(id="10", x=0, y=12), Landmark(id="11", x=0, y=16), Landmark(id="12", x=4, y=-4), Landmark(id="13", x=4, y=0), Landmark(id="14", x=4, y=4), Landmark(id="15", x=4, y=8), Landmark(id="16", x=4, y=12), Landmark(id="17", x=4, y=16), Landmark(id="18", x=8, y=-4), Landmark(id="19", x=8, y=0), Landmark(id="20", x=8, y=4), Landmark(id="21", x=8, y=8), Landmark(id="22", x=8, y=12), Landmark(id="23", x=8, y=16), Landmark(id="24", x=12, y=-4), Landmark(id="25", x=12, y=0), Landmark(id="26", x=12, y=4), Landmark(id="27", x=12, y=8), Landmark(id="28", x=12, y=12), Landmark(id="29", x=12, y=16), Landmark(id="30", x=16, y=-4), Landmark(id="31", x=16, y=0), Landmark(id="32", x=16, y=4), Landmark(id="33", x=16, y=8), Landmark(id="34", x=16, y=12), Landmark(id="35", x=16, y=16)), num_particles=100, particles=LocalizationParticles(x=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), y=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), theta=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))), [Measurement(landmark_id="35", r=22.753147219062917, b=0.7738698392857971), Measurement(landmark_id="25", r=12.640422650443282, b=0.009154262150241616), Measurement(landmark_id="15", r=8.408602536838048, b=1.1387038391295914), Measurement(landmark_id="3", r=10.248271955129297, b=2.117092341243524), Measurement(landmark_id="32", r=15.78868726666365, b=0.1345498076509456), Measurement(landmark_id="12", r=5.033579786955029, b=-0.7817917913663452), Measurement(landmark_id="7", r=-2.3250307746388343, b=-0.01909317455769924), Measurement(landmark_id="20", r=7.698360962746094, b=0.399745227278584), Measurement(landmark_id="5", r=15.948163519613333, b=1.7881725941556874), Measurement(landmark_id="2", r=6.068484785866513, b=2.4471709441565186), Measurement(landmark_id="26", r=12.520575977729484, b=0.44099698774324514), Measurement(landmark_id="24", r=11.983915967186904, b=-0.291075514011446), Measurement(landmark_id="8", r=4.903470181651809, b=1.5790004474613593), Measurement(landmark_id="30", r=15.748923253116834, b=-0.32541437620132907), Measurement(landmark_id="11", r=15.542274174332661, b=1.5900119763018357), Measurement(landmark_id="4", r=11.639492457134782, b=1.8742928687102223), Measurement(landmark_id="34", r=19.84077499008552, b=0.6906987897814902), Measurement(landmark_id="9", r=8.21465912250634, b=1.6018084460172888), Measurement(landmark_id="27", r=13.76837649243762, b=0.5766916802305041), Measurement(landmark_id="14", r=6.44082971955371, b=0.9157246165771075), Measurement(landmark_id="23", r=16.629478287894198, b=1.2392634902611783), Measurement(landmark_id="21", r=12.659583922767066, b=0.8535804455254041), Measurement(landmark_id="13", r=4.264455630329303, b=-0.027394933553940907), Measurement(landmark_id="1", r=5.458020683536959, b=3.312657462648057), Measurement(landmark_id="6", r=5.801634869866125, b=-1.4560318732949584), Measurement(landmark_id="33", r=18.245924230657273, b=0.3582019162682134), Measurement(landmark_id="18", r=8.939817776879076, b=-0.40635936363131575), Measurement(landmark_id="0", r=4.368492785742826, b=-2.3217135857051487), Measurement(landmark_id="31", r=16.42986369482223, b=0.06074118633851044), Measurement(landmark_id="22", r=13.23808713509877, b=0.9250492804923485), Measurement(landmark_id="19", r=7.563564752856779, b=-0.10208447443373762), Measurement(landmark_id="10", r=13.739367877130134, b=1.5275199517585014), Measurement(landmark_id="16", r=12.97808027013372, b=1.2264810610293408), Measurement(landmark_id="28", r=18.554035627279266, b=0.9006213960947502), Measurement(landmark_id="29", r=20.633352622824916, b=0.7350027054140555), Measurement(landmark_id="17", r=16.54445147673053, b=1.3854805390653757)], array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01]))
print('Tests passed.')

## Localization: Particle Filter Update


### Question

What we want you to do is to write a method for step 4: `update` that takes in a
set of samples, the most recent command, the most recent measurement, and the
motion and sensor covariances and does the following:
  * use the motion_model method to update the sample based on the command
  * use compute_weights to compute the sample weights from the
measurement,
  * resample a new set of samples,
  * blur the samples by applying noise drawn from a zero-mean Gaussian with
covariance equal to `np.diag(.001, .001, np.deg2rad(1)**2)`. (You'll need to
recall from lecture why we do this blurring --- if you have machine learning
background, it's a form of regularization, but you don't need to know about
regularization to understand why this is important.)


For reference, our solution is **16** line(s) of code.

In [ ]:
@implementation_for(Localization)
def update(self: Localization, command: Command,
           measurements: Sequence[Measurement]) -> None:
    """Update the samples, based on the command and the new measurement.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
        movement, and delta_theta(float), the rotation of the movement.
        measurement: a measurement vector. The measurement is computed by
            `simulator.simulate_sensing` function.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def sampling_problem3_test(_, infer: Localization, command: Command,
                           measurements: Sequence[Measurement], pose: Pose):
    infer.update(command, measurements)
    est_pose = infer.estimated_pose()
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=0.3), "Estimated pose should be quite close to our estimation"

sampling_problem3_test(update, Localization(motion_noise_covariance=array([[0.0001, 0.    ],
       [0.    , 0.0001]]), sensor_noise_covariance=array([[1.00000000e-06, 0.00000000e+00],
       [0.00000000e+00, 9.27917724e-08]]), landmarks=(Landmark(id="0", x=-4, y=-4), Landmark(id="1", x=-4, y=0), Landmark(id="2", x=-4, y=4), Landmark(id="3", x=-4, y=8), Landmark(id="4", x=-4, y=12), Landmark(id="5", x=-4, y=16), Landmark(id="6", x=0, y=-4), Landmark(id="7", x=0, y=0), Landmark(id="8", x=0, y=4), Landmark(id="9", x=0, y=8), Landmark(id="10", x=0, y=12), Landmark(id="11", x=0, y=16), Landmark(id="12", x=4, y=-4), Landmark(id="13", x=4, y=0), Landmark(id="14", x=4, y=4), Landmark(id="15", x=4, y=8), Landmark(id="16", x=4, y=12), Landmark(id="17", x=4, y=16), Landmark(id="18", x=8, y=-4), Landmark(id="19", x=8, y=0), Landmark(id="20", x=8, y=4), Landmark(id="21", x=8, y=8), Landmark(id="22", x=8, y=12), Landmark(id="23", x=8, y=16), Landmark(id="24", x=12, y=-4), Landmark(id="25", x=12, y=0), Landmark(id="26", x=12, y=4), Landmark(id="27", x=12, y=8), Landmark(id="28", x=12, y=12), Landmark(id="29", x=12, y=16), Landmark(id="30", x=16, y=-4), Landmark(id="31", x=16, y=0), Landmark(id="32", x=16, y=4), Landmark(id="33", x=16, y=8), Landmark(id="34", x=16, y=12), Landmark(id="35", x=16, y=16)), num_particles=100, particles=LocalizationParticles(x=array([0.98116564, 0.98374857, 0.9804707 , 0.98030241, 0.97996945,
       0.98033665, 0.97910682, 0.98109879, 0.98100137, 0.98247916,
       0.98349377, 0.98151862, 0.98089557, 0.98084217, 0.98283572,
       0.98151999, 0.9839135 , 0.98142974, 0.98103791, 0.98255549,
       0.98140029, 0.98350079, 0.98209223, 0.98065804, 0.9827234 ,
       0.98049622, 0.98254329, 0.98074784, 0.98077212, 0.98079254,
       0.98158725, 0.9825394 , 0.98133293, 0.98144326, 0.98124833,
       0.9817101 , 0.98125886, 0.98169492, 0.98139318, 0.9813157 ,
       0.98024684, 0.98136911, 0.98185518, 0.98286645, 0.98006929,
       0.98040044, 0.98047605, 0.98213946, 0.97937825, 0.9817623 ,
       0.98157254, 0.98118723, 0.98026057, 0.98432512, 0.98241912,
       0.98136399, 0.98106577, 0.98122163, 0.98156954, 0.9810925 ,
       0.98209289, 0.98029388, 0.98083749, 0.98075291, 0.98029706,
       0.98027498, 0.98042083, 0.9818858 , 0.9818174 , 0.98273301,
       0.98084645, 0.98242155, 0.98241331, 0.98368051, 0.98153205,
       0.98188449, 0.98055181, 0.98207794, 0.9791555 , 0.9819787 ,
       0.98088237, 0.98255869, 0.97988158, 0.98191864, 0.98102001,
       0.98112082, 0.98168518, 0.98120931, 0.97960413, 0.98209027,
       0.98148121, 0.98139401, 0.97999712, 0.98297606, 0.98140074,
       0.98305921, 0.98079911, 0.98140514, 0.98073764, 0.98098068]), y=array([-8.56549308e-04, -4.04032294e-04,  3.67544896e-04,  1.82272360e-03,
        9.60693398e-04, -1.40134729e-03, -1.22662166e-03, -2.63937349e-04,
       -1.29868672e-03,  2.05332564e-04, -2.76432845e-04,  1.47132196e-03,
        3.08331246e-04, -8.40290395e-04, -7.92286662e-04,  1.29782527e-03,
       -7.46679783e-05,  1.43504939e-03, -9.47488595e-04, -4.10381794e-04,
        8.63519658e-04, -1.27916111e-03,  2.96733172e-04, -7.48049827e-05,
        6.90429024e-04,  9.68882639e-04,  2.01406015e-03, -8.64044991e-04,
        3.59504400e-04,  1.06458514e-03, -4.05941727e-04, -3.03098253e-04,
       -7.99422400e-04,  4.12870820e-04, -1.14761094e-03,  8.92473887e-04,
        2.28053325e-04, -1.81807763e-03, -1.00033035e-03,  1.56340475e-03,
        2.24252221e-04,  9.72192320e-05, -4.17477350e-04, -7.92115021e-04,
        1.11730532e-03,  1.37689826e-04,  7.53991467e-04,  7.66919670e-04,
        1.66925081e-03,  1.11496232e-03,  4.35546159e-04, -8.54161261e-04,
       -1.56023891e-04,  1.89499638e-04, -3.00783876e-04, -1.24062460e-03,
        9.83779068e-05,  1.63380841e-05, -1.03148246e-03, -1.43273549e-03,
        6.07111672e-04,  3.28301295e-04,  5.96906481e-04, -8.00082476e-04,
        6.77462169e-04, -2.31101608e-04,  2.14957453e-03, -6.54883751e-05,
       -1.10461660e-03, -7.95500550e-04,  2.69024073e-04,  4.99498233e-04,
        7.04111022e-04,  1.76992139e-03,  1.32435875e-03,  1.10457847e-03,
        9.21408398e-04, -1.32568015e-04, -1.14600043e-03,  5.31178367e-04,
        2.17179633e-04,  1.46440770e-04,  1.51875934e-03, -2.68372735e-04,
        1.22138496e-03, -1.53080350e-03,  2.39145581e-04,  2.98977456e-04,
       -4.55825348e-04,  6.78380099e-04,  1.63159743e-03,  1.58958674e-04,
        7.70673054e-04, -7.56504706e-05, -6.25557035e-04, -1.47772197e-05,
       -1.89236189e-03,  1.05475793e-03, -2.76822995e-04, -5.10029540e-04]), theta=array([0.76547154, 0.76549586, 0.7659991 , 0.76578225, 0.76634599,
       0.76625502, 0.76623583, 0.76604861, 0.76632985, 0.76595488,
       0.76586206, 0.76642632, 0.76552134, 0.76563443, 0.7657792 ,
       0.76608769, 0.76601994, 0.76609565, 0.76601559, 0.76610226,
       0.76569599, 0.76582978, 0.76619396, 0.76594486, 0.76618381,
       0.76600985, 0.76584769, 0.7658974 , 0.76589709, 0.76565544,
       0.76616179, 0.76607548, 0.76601448, 0.7658807 , 0.76583204,
       0.76581249, 0.76600251, 0.76592611, 0.76645106, 0.76570033,
       0.76542977, 0.76625052, 0.76560682, 0.76562265, 0.76578307,
       0.76587809, 0.76626565, 0.76606378, 0.76603312, 0.76637776,
       0.7657586 , 0.76572183, 0.7662607 , 0.76553038, 0.7657399 ,
       0.7660089 , 0.76605732, 0.76602774, 0.76550553, 0.7659678 ,
       0.76562184, 0.76581889, 0.76564039, 0.76562338, 0.76595692,
       0.76544192, 0.76591364, 0.76604719, 0.76593171, 0.76576859,
       0.76610103, 0.76592223, 0.76548552, 0.76588989, 0.76588868,
       0.765632  , 0.7660815 , 0.7658531 , 0.76595547, 0.76590207,
       0.76596496, 0.76548992, 0.76558438, 0.76588942, 0.76588239,
       0.76600409, 0.76636696, 0.7664433 , 0.76637557, 0.76619477,
       0.7660562 , 0.76599988, 0.7659014 , 0.76606935, 0.76624335,
       0.7658496 , 0.76588697, 0.76623358, 0.76628349, 0.76636532]))), Command(delta_p=1, delta_theta=0.7853981633974483), [Measurement(landmark_id="4", r=12.649236370894611, b=1.8925066397489418), Measurement(landmark_id="16", r=12.649751063323961, b=1.249077726801277), Measurement(landmark_id="20", r=8.943736240625999, b=0.46375775715344025), Measurement(landmark_id="35", r=22.628720998014654, b=0.7856866607567657), Measurement(landmark_id="34", r=19.999296264764194, b=0.6431156393698316), Measurement(landmark_id="10", r=11.999376725537463, b=1.5708089154080955), Measurement(landmark_id="12", r=5.654529218717742, b=-0.7854648111495861), Measurement(landmark_id="7", r=-0.0012459109472530653, b=-0.00022306139218380394), Measurement(landmark_id="33", r=17.887999561015462, b=0.4635512584632948), Measurement(landmark_id="17", r=16.492834133007015, b=1.3261352314007253), Measurement(landmark_id="8", r=3.999871465337056, b=1.5712125753715283), Measurement(landmark_id="31", r=15.999334805326514, b=0.00010707609058078074), Measurement(landmark_id="1", r=4.000903470181652, b=3.141621291373365), Measurement(landmark_id="29", r=19.999256500750647, b=0.9270144443957444), Measurement(landmark_id="15", r=8.943814184173492, b=1.1072157930644515), Measurement(landmark_id="5", r=16.491412884287104, b=1.815711271397861), Measurement(landmark_id="18", r=8.944112684989244, b=-0.46348285801429606), Measurement(landmark_id="25", r=12.000214659122506, b=0.00010825271769035358), Measurement(landmark_id="23", r=17.8878899913889, b=1.1071092352234257), Measurement(landmark_id="21", r=11.314492474454822, b=0.7858530885395344), Measurement(landmark_id="11", r=15.998740934467897, b=1.5712574943489113), Measurement(landmark_id="32", r=16.493768377894426, b=0.24521666418979546), Measurement(landmark_id="28", r=16.970827204107472, b=0.7853025370396719), Measurement(landmark_id="30", r=16.49388052315418, b=-0.24438153429639173), Measurement(landmark_id="27", r=14.424006736725824, b=0.5884032070631326), Measurement(landmark_id="3", r=8.944629290409818, b=2.0340758608916567), Measurement(landmark_id="22", r=14.422200647722839, b=0.9829936969482067), Measurement(landmark_id="0", r=5.655565888028631, b=-2.3560741291298712), Measurement(landmark_id="14", r=5.6572841131872025, b=0.785610190136083), Measurement(landmark_id="24", r=12.647926522706761, b=-0.32195212052680294), Measurement(landmark_id="2", r=5.656417814245238, b=2.3558381481535373), Measurement(landmark_id="6", r=4.00173936787713, b=-1.5709473898414392), Measurement(landmark_id="9", r=8.00032896962946, b=1.5707175610932667), Measurement(landmark_id="19", r=8.001583472878803, b=0.00040220495707190147), Measurement(landmark_id="26", r=12.649743993296344, b=0.3210793269023251), Measurement(landmark_id="13", r=4.00005202897426, b=0.00020826272337815132)], Pose(x=0.9858299248258179, y=-0.00020686081759629056, theta=0.768264535456641))
print('Tests passed.')

## SLAM: Update by Motion Model


### Question

Please implement a function that takes in a control command $[\Delta p_t, \Delta \theta_t]$ 
and updates the particles by drawing drawn from the posterior $X_{t+1}$ using the 
motion model. 

This function behaves the same as `Localization.motion_model` for the pose 
components of the particles, and simply keeps the landmarks components 
(`RBSLAMParticles.landmarks_loc`) of the particles the same.


For reference, our solution is **12** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def motion_model(self: SLAM, command: Command):
    """A motion model that simulates a one-step movement of the robot, and
    returns the new samples. TODO(camyang): fix docstring

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
            movement, and delta_theta(float), the rotation of the movement.

    Returns:
        new_samples: a batch of new samples obtained by applying the motion model. 
    """
    raise NotImplementedError("Implement me!")

## SLAM: Tracking New Landmarks


### Question

Since our robot might have limited sensing range (see the parameter `Simulator.max_sensing_range`),
the robot is not aware of all the possible landmarks out there in the world.
Therefore, we must track new landmarks that appears in the measurements in our 
particles.

One way to handle missing data due to sensing range is to explicitly model the 
sensing range in our probabilistic model of the sensors. 
Howewer, this approach would induce a difficult inference problem.  
Instead, we will take the approach of treating the missingness as uninformative.
In particular, we will consider sensor data within-range as observed variable, 
and sensor data out-of-range as unobserved variables.
We will assume that each new landmark follows a multivariate normal distribution
centered at the location of its first occurence, with a high enough covariance 
`self.new_landmarks_init_covariance`.

Please implement a function that takes a set of measurements and updates the 
particles to include potentially new landmarks that are not yet tracked by the 
particles. 

Below we give a description for this procedure:

- Filter the input measurements to contain only measurements to the new (untracked) 
landmarks. 
- For each new landmark measurement and each particle:
    - Find the predicted location using the robot's pose of that particle and the landmark's measurement.
    - Add a Gaussian noise of covariance `self.new_landmarks_init_covariance` to this predicted location.
    - This nosiy location becomes the landmark's location in the current particle.
- Update `self.particles.landmarks_id_to_idx` to maintain the invariant that it 
holds a mapping from each tracked unique landmark indentifier to the index of 
that landmark in `self.particles.landmarks_loc`. 


For reference, our solution is **31** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def add_new_landmarks(self: SLAM, measurements: Sequence[Measurement]) -> None:
    """Add new landmarks into all particles to track them.

    Args:
        measurements: the measurements. This function will add only the new
        landmarks in these measurements (i.e., those that had not been added 
        before).
    """
    raise NotImplementedError("Implement me!")

## SLAM: Computing the Importance Weights


### Question

Please implement the `SLAM.compute_weights` function that takes a 
sequence of measurements and returns the importance weights of the particles.

This function is similar to `Localization.compute_weights`, except that it takes 
into account the landmarks location tracked in the particles.


For reference, our solution is **30** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def compute_weights(self: SLAM,
                    measurements: Sequence[Measurement]) -> np.ndarray:
    """Compute the importance weights of the samples, based on the new
    measurement. TODO(camyang): fix docstring

    Args:
        measurements: a sequence of measurements made by the robot.

    Returns:
        weights: a numpy array of importance weights, normalized to 1.
    """
    raise NotImplementedError("Implement me!")

## SLAM: Particle Filter Update


### Question

Please implement the update step of SLAM.

The procedure is similar to `Localization.update`, except that:
- It does an additional `add_new_landmarks` step after forward updating with the `motion_model` 
and before `compute_weights`. 
- It blur the _measured_ landmarks' locations by applying noise drawn from a zero-mean Gaussian with
a covariance of `np.diag([.001, .001])`. 


For reference, our solution is **29** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def update(self: SLAM, command: Command,
           measurements: Sequence[Measurement]) -> None:
    """Update the samples, based on the command and the new measurement.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
        movement, and delta_theta(float), the rotation of the movement.
        measurement: a measurement vector. The measurement is computed by
            `simulator.simulate_sensing` function.
    """
    raise NotImplementedError("Implement me!")

## Einsum Warmup (Optional)


### Question

This question intends to get you familiar with using `numpy.einsum`.
**This question is optional and you do not have to use `einsum` in your implementation.** 
However, using `einsum` can greatly simplify your implementation.

In particular, the Rao-blackwellised particle filter performs quite a few 
matrix multiplications. 
The function `numpy.einsum` allows one to express batched multiplication of 
multiple matrices succinctly.

If you have not familar with `np.einsum`, here are some guides online:
- [StackOverflow Post](https://stackoverflow.com/questions/26089893/understanding-numpys-einsum)
- [Blog: einsum is all you need](https://rockt.github.io/2018/04/30/einsum)

For this warmup question, please fill in the indexing expression string for 
`einsum` to implement a 'sandwich matrix product'. 
A sandwich matrix product of two matrices $U$ and $S$ is
$M = U \cdot S \cdot U^\top$.


For reference, our solution is **2** line(s) of code.

In [ ]:
def sandwich_matrix_product(U: np.ndarray, S: np.ndarray) -> np.ndarray:
    """Perform a batched 'sandwitch matrix product': $$M = U \cdot S \cdot U^\top$$.
    Uses `numpy.einsum`.

    Args:
        U, S: input matrices to perform the product, both of shape (batch, N, N)

    Returns:
        the matrix M.
    """
    return np.einsum("FILL IN THE STRING HERE", U, S, U)

### Tests

In [ ]:

assert np.allclose(sandwich_matrix_product(np.array([[[-0.91509102,  0.02896068,  0.09373636],   [-0.31979851, -0.86280633, -0.5421848 ],   [-0.28403213, -0.12971603,  0.18185345]],   [[ 0.44478304, -0.36473625, -0.34209248],   [-0.96061671, -0.91825028, -0.48435661],   [ 0.48049   ,  0.25662766,  0.53957804]],   [[ 0.53783887,  0.71313494,  0.44063853],   [ 0.95802184,  0.79765044,  0.17343433],   [ 0.17631534, -0.93146592,  0.99705316]],   [[-0.73684801,  0.48069439,  0.64203039],   [-0.25389094, -0.60629589, -0.80248023],   [ 0.49721201, -0.09469294,  0.42743552]],   [[ 0.8308153 , -0.70683253,  0.838342  ],   [-0.17674708, -0.38946598,  0.88612452],   [ 0.98130339, -0.60221556,  0.31367669]]], dtype=np.float64), np.array([[[-0.78700937,  0.30182801,  0.65462646],   [ 0.36899709, -0.16533372, -0.23386728],   [-0.21375517,  0.17942364,  0.76313454]],   [[ 0.85813231, -0.89294076, -0.63675521],   [-0.77555137, -0.61333072, -0.30678438],   [ 0.01306337,  0.25892245,  0.46428444]],   [[ 0.78022308,  0.97817687,  0.32571296],   [ 0.69072904,  0.55607769, -0.38493592],   [ 0.75138454, -0.91447372, -0.99926531]],   [[-0.45253474, -0.07580494,  0.27672579],   [-0.79645947,  0.34602027,  0.60363173],   [-0.62937416, -0.16974949,  0.03996998]],   [[-0.09638596,  0.59965986,  0.9210448 ],   [ 0.59790633, -0.84401436,  0.60987114],   [-0.86680734, -0.52805923, -0.69380621]]], dtype=np.float64)), np.array([[[-0.70821093,  0.29028311, -0.264794  ],   [-0.0860032 ,  0.25683661, -0.0839603 ],   [-0.12767688, -0.00731765, -0.0378081 ]],   [[ 0.5021111 , -0.23356351,  0.07177661],   [-0.75393822, -1.39959106,  0.63624027],   [ 0.30684435,  0.26353458, -0.08116496]],   [[ 0.80153175,  1.51206662, -0.80293068],   [ 1.56577159,  2.31436395, -1.05422099],   [-1.02144813, -0.87038954,  0.63539018]],   [[ 0.46041849, -0.04331244, -0.18835761],   [-1.13941324,  0.12874572,  0.35331832],   [ 0.30230967,  0.11473338, -0.15291035]],   [[-1.68979947, -0.25599802, -1.6767293 ],   [-1.49516034, -0.63011489, -1.13628625],   [-0.99438737,  0.04839135, -1.17364301]]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

## RB SLAM: Update by Motion Model


### Question

Please implement a function that takes in a control command $[\Delta p_t, \Delta \theta_t]$ 
and updates the particles by drawing drawn from the posterior $X_{t+1}$ using the 
motion model. 

This function behaves the same as `Localization.motion_model` for the pose 
components of the particles, and simply keeps the landmarks components 
(`RBSLAMParticles.landmarks_loc` and `RBSLAMParticles.landmarks_cov`) of the 
particles the same.


For reference, our solution is **9** line(s) of code.

In [ ]:
@implementation_for(RBSLAM)
def motion_model(self: RBSLAM, command: Command) -> None:
    """A motion model that simulates a one-step movement of the robot, and
    returns the new samples. TODO(camyang): fix docstring

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
            movement, and delta_theta(float), the rotation of the movement.

    Returns:
        new_samples: a batch of new samples obtained by applying the motion model. 
    """
    raise NotImplementedError("Implement me!")

## RBSLAM: Tracking New Landmarks


### Question

Since our robot might have limited sensing range (see the parameter `Simulator.max_sensing_range`),
the robot is not aware of all the possible landmarks out there in the world.
Therefore, we must track new landmarks that appears in the measurements in our 
particles.

We will make the same assumaption as in `SLAM.add_new_landmarks`:
We assume that each new landmark follows a multivariate normal distribution
centered at the location of its first occurence, with a high enough covariance 
`self.new_landmarks_init_covariance`.

Please implement `add_new_landmarks` that takes a sequence of measurements and updates 
the particles to include potentially new landmarks that are not yet tracked by the 
particles. 

Below we give a description for this procedure:

- Filter the input measurements to contain only measurements to the new (untracked) 
landmarks. 
- For each new landmark measurement and each particle:
    - Find the predicted location using the robot's pose of that particle and the landmark's measurement.
    - Add the new landmark into the current particle, represented as a multivariate Gaussian centered at 
    the predicted location and a covaraince of `self.new_landmarks_init_covariance`. 
- Update `self.particles.landmarks_id_to_idx` to maintain the invariant that it 
holds a mapping from each tracked unique landmark indentifier to the index of 
that landmark in `self.particles.landmarks_loc`. 


For reference, our solution is **31** line(s) of code.

In [ ]:
@implementation_for(RBSLAM)
def add_new_landmarks(self, measurements: Sequence[Measurement]) -> None:
    """Add new landmarks into all particles to track them.

    Args:
        measurements: the measurements. This function will add only the new
        landmarks in these measurements (i.e., those that had not been added 
        before).
    """
    raise NotImplementedError("Implement me!")

## RB SLAM: Extended Kalman Filter Update


### Question

Please implement `update_EKF` that takes a sequence of measurements and performs
an Extended Kalman Filter update on the landmarks of the particles.


For reference, our solution is **36** line(s) of code.

In [ ]:
@implementation_for(RBSLAM)
def update_EKF(self, measurements: Sequence[Measurement]) -> None:
    """Performs an extended Kalman filter update on the landmarks.

    Args:
        measurements: the sequence of landmark measurements.
    """
    raise NotImplementedError("Implement me!")

## RB SLAM: Computing the Importance Weights


### Question

Please implement a function that takes a set of samples and measurements and
returns the importance weights of the samples. 


For reference, our solution is **44** line(s) of code.

In [ ]:
@implementation_for(RBSLAM)
def compute_weights(self: RBSLAM,
                    measurements: Sequence[Measurement]) -> np.ndarray:
    """Compute the importance weights of the particles, based on the new
    measurement.

    Args:
        measurements: a sequence of measurements made by the robot.

    Returns:
        weights: a numpy array of importance weights, normalized to 1.
    """
    raise NotImplementedError("Implement me!")

## RB SLAM: Particle Filter Update


### Question

Please implement the update step of SLAM.

The procedure is similar to `SLAM.update`, except:
- It does an additional `update_EKF` step after `add_new_landmarks` and before `compute_weights`. 
- Instead of blurring the landmarks' locations for regularization, it adds a diagonal covariance
`np.diag([0.001, 0.001])` to all _measured_ landmarks' covariances.


For reference, our solution is **31** line(s) of code.

In [ ]:
@implementation_for(RBSLAM)
def update(self: RBSLAM, command: Command,
           measurements: Sequence[Measurement]) -> None:
    """Update the samples, based on the command and the new measurement.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
        movement, and delta_theta(float), the rotation of the movement.
        measurement: a measurement vector. The measurement is computed by
            `simulator.simulate_sensing` function.

    Returns:
        new_samples: a numpy array of updated samples.
    """
    raise NotImplementedError("Implement me!")